In [1]:
import re
import pandas as pd
import sys
import os
import json
from tqdm import tqdm

from allennlp.common import JsonDict

sys.path.insert(0, '..')

import utils.semantic_role_labeling as sem_rl

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

def _run_predictor(batch_data):
    if len(batch_data) == 1:
        result = predictor.predict_json(batch_data[0])
        results = [result]
    else:
        results = predictor.predict_batch_json(batch_data)
    return results


# Read ROCStories into pandas DataFrame
roc_stories_path_csv = '../../generated/coreference_resolution/ROCStories_with_resolved_coreferences.csv'
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep='\t', header=0)
len(roc_stories_df)

52665

In [2]:
roc_stories_df = roc_stories_df.iloc[:1000]

In [3]:
predictor = sem_rl.get_predictor()
batch_size = 128

for n in range(1, 6):
    batch_data = []
    batch_result = []

    for index, row in tqdm(roc_stories_df.iterrows()):
        line = str(row[f'resolved{n}'])
        if not line.isspace():
            line = {"sentence": line.strip()}
            line = json.dumps(line)
            json_data = predictor.load_line(line)
            batch_data.append(json_data)
            if len(batch_data) == batch_size:
                res = _run_predictor(batch_data)
                for b in res:
                    batch_result.append(b)
                batch_data = []

    if batch_data:
        res = _run_predictor(batch_data)
        for b in res:
            batch_result.append(b)

    roc_stories_df[f'srl_r{n}'] = batch_result

FileNotFoundError: file ../data/pre-trained-transformer-srl/srl_bert_base_conll2012.tar.gz not found

In [ ]:
len(batch_result)

In [ ]:
roc_stories_df.loc[0, 'srl_r1']